## Grid Objectives
Iterating between min and max for each column

### Glossary
- **task**: Refers to the set of values (row) and corresponding keys to be aimed at sequentially.
- **objective**: Refers to one key (column) and respective value to be aimed at simultaneously during a task.
- **experiment**: Refers to one file containing a multiple of objectives and tasks for a fixed number of each, respectively. 

In [1]:
import itertools
import json
import numpy as np
import os
import pandas as pd

In [2]:
#Features between 0 and 1: 
normalized_feature_names = ['ratio_variants_per_number_of_traces', 'trace_len_hist1', 'trace_len_hist2',
                            'trace_len_hist3', 'trace_len_hist4', 'trace_len_hist5', 'trace_len_hist7',
                            'trace_len_hist8', 'trace_len_hist9', 'ratio_most_common_variant', 
                            'ratio_top_1_variants', 'ratio_top_5_variants', 'ratio_top_10_variants', 
                            'ratio_top_20_variants', 'ratio_top_50_variants', 'ratio_top_75_variants', 
                            'epa_normalized_variant_entropy', 'epa_normalized_sequence_entropy', 
                            'epa_normalized_sequence_entropy_linear_forgetting', 'epa_normalized_sequence_entropy_exponential_forgetting']

normalized_feature_names = ['ratio_variants_per_number_of_traces', 'ratio_most_common_variant', 
                            'ratio_top_10_variants', 'epa_normalized_variant_entropy', 'epa_normalized_sequence_entropy', 
                            'epa_normalized_sequence_entropy_linear_forgetting', 'epa_normalized_sequence_entropy_exponential_forgetting']

def abbrev_obj_keys(obj_keys):
    abbreviated_keys = []
    for obj_key in obj_keys:
        key_slices = obj_key.split("_")
        chars = []
        for key_slice in key_slices:
            for idx, single_char in enumerate(key_slice):
                if idx == 0 or single_char.isdigit():
                    chars.append(single_char)
        abbreviated_key = ''.join(chars)
        abbreviated_keys.append(abbreviated_key)
    return '_'.join(abbreviated_keys) 

In [3]:
def write_generator_experiment(experiment_path, objectives=["ratio_top_20_variants", "epa_normalized_sequence_entropy_linear_forgetting"]):
    first_dir = os.path.split(experiment_path[3:])[-1].replace(".csv","")
    second_dir = first_dir.replace("grid_","").replace("objectives","")

    experiment = [
      {
        'pipeline_step': 'event_logs_generation',
        'output_path':'output/generated/grid_2obj',
        'generator_params': {
          "experiment": {"input_path": experiment_path[3:],
            "objectives": objectives},
          'config_space': {
            'mode': [5, 20],
            'sequence': [0.01, 1],
            'choice': [0.01, 1],
            'parallel': [0.01, 1],
            'loop': [0.01, 1],
            'silent': [0.01, 1],
            'lt_dependency': [0.01, 1],
            'num_traces': [10, 10001],
            'duplicate': [0],
            'or': [0]
          },
          'n_trials': 200
        }
      },
      {
        'pipeline_step': 'feature_extraction',
        'input_path': os.path.join('output','features', 'generated', 'grid_2obj', first_dir, second_dir),
        "feature_params": {"feature_set":["ratio_variants_per_number_of_traces","ratio_most_common_variant","ratio_top_10_variants","epa_normalized_variant_entropy","epa_normalized_sequence_entropy","epa_normalized_sequence_entropy_linear_forgetting","epa_normalized_sequence_entropy_exponential_forgetting"]},
        'output_path': 'output/plots',
        'real_eventlog_path': 'data/BaselineED_feat.csv',
        'plot_type': 'boxplot'
      },
      {
        "pipeline_step": "benchmark_test",
        "benchmark_test": "discovery",
        "input_path": os.path.join('output', 'generated', 'grid_2obj', first_dir, second_dir),
        "output_path":"output",
        "miners" : ["heu", "imf", "ilp"]
      }
    ]

    #print("EXPERIMENT:", experiment[1]['input_path'])
    output_path = os.path.join('..', 'config_files','algorithm','grid_2obj')
    os.makedirs(output_path, exist_ok=True)
    output_path = os.path.join(output_path, f'generator_{os.path.split(experiment_path)[-1].split(".")[0]}.json') 
    with open(output_path, 'w') as f:
        json.dump(experiment, f, ensure_ascii=False)
    print(f"Saved experiment config in {output_path}")
    
    return experiment

def create_objectives_grid(objectives, n_para_obj=2):
    parameters_o = "objectives, "
    if n_para_obj==1:
        experiments = [[exp] for exp in objectives]
    else:
        experiments = eval(f"[exp for exp in list(itertools.product({(parameters_o*n_para_obj)[:-2]})) if exp[0]!=exp[1]]")
    experiments = list(set([tuple(sorted(exp)) for exp in experiments]))
    print(len(experiments), experiments)
    
    parameters = "np.around(np.arange(0, 1.1,0.1),2), "
    tasks = eval(f"list(itertools.product({(parameters*n_para_obj)[:-2]}))")
    tasks = [(f'task_{i+1}',)+task for i, task in enumerate(tasks)]
    print(len(tasks))
    for exp in experiments:
        df = pd.DataFrame(data=tasks, columns=["task", *exp])
        experiment_path = os.path.join('..','data', 'grid_2obj')
        os.makedirs(experiment_path, exist_ok=True)
        experiment_path = os.path.join(experiment_path, f"grid_{len(df.columns)-1}objectives_{abbrev_obj_keys(exp)}.csv") 
        df.to_csv(experiment_path, index=False)
        print(f"Saved experiment in {experiment_path}")
        write_generator_experiment(experiment_path, objectives=exp)
    #df.to_csv(f"../data/grid_{}objectives_{abbrev_obj_keys(objectives.tolist())}.csv" ,index=False)
        
exp_test = create_objectives_grid(normalized_feature_names, n_para_obj=2)        
print(exp_test)

21 [('epa_normalized_sequence_entropy_linear_forgetting', 'ratio_most_common_variant'), ('ratio_top_10_variants', 'ratio_variants_per_number_of_traces'), ('epa_normalized_sequence_entropy_linear_forgetting', 'ratio_variants_per_number_of_traces'), ('epa_normalized_variant_entropy', 'ratio_most_common_variant'), ('epa_normalized_variant_entropy', 'ratio_variants_per_number_of_traces'), ('epa_normalized_sequence_entropy_linear_forgetting', 'ratio_top_10_variants'), ('epa_normalized_sequence_entropy', 'epa_normalized_variant_entropy'), ('epa_normalized_sequence_entropy_exponential_forgetting', 'epa_normalized_variant_entropy'), ('epa_normalized_sequence_entropy', 'ratio_most_common_variant'), ('epa_normalized_variant_entropy', 'ratio_top_10_variants'), ('epa_normalized_sequence_entropy_exponential_forgetting', 'ratio_most_common_variant'), ('epa_normalized_sequence_entropy', 'ratio_variants_per_number_of_traces'), ('epa_normalized_sequence_entropy_exponential_forgetting', 'ratio_variants_

### Helper prototypes

In [4]:
df = pd.DataFrame(columns=["log","ratio_top_20_variants", "epa_normalized_sequence_entropy_linear_forgetting"])    

In [5]:
k=0
for i in np.arange(0, 1.1,0.2):
    for j in np.arange(0,0.55,0.1):
        k+=1
        new_entry = pd.Series({'log':f"objective_{k}", "ratio_top_20_variants":round(i,1),
                   "epa_normalized_sequence_entropy_linear_forgetting":round(j,1)})
        df = pd.concat([
                df, 
                pd.DataFrame([new_entry], columns=new_entry.index)]
           ).reset_index(drop=True)
    

In [6]:
df.to_csv("../data/grid_objectives.csv" ,index=False)

## Objectives from real logs
(Feature selection)

In [7]:
bpic_features = pd.read_csv("../data/BaselineED_feat.csv", index_col=None)
#bpic_features = pd.read_csv("../gedi/output/features/real_event_logs.csv", index_col=None)

#bpic_features = bpic_features.drop(['Unnamed: 0'], axis=1)
print(bpic_features.shape)
print(len(bpic_features), " Event-Logs: ", bpic_features.sort_values('log')['log'].unique())

#bpic_features.rename(columns={"variant_entropy":"epa_variant_entropy", "normalized_variant_entropy":"epa_normalized_variant_entropy", "sequence_entropy":"epa_sequence_entropy", "normalized_sequence_entropy":"epa_normalized_sequence_entropy", "sequence_entropy_linear_forgetting":"epa_sequence_entropy_linear_forgetting", "normalized_sequence_entropy_linear_forgetting":"epa_normalized_sequence_entropy_linear_forgetting", "sequence_entropy_exponential_forgetting":"epa_sequence_entropy_exponential_forgetting", "normalized_sequence_entropy_exponential_forgetting":"epa_normalized_sequence_entropy_exponential_forgetting"},
#          errors="raise", inplace=True)

bpic_features.head()
#bpic_features.to_csv("../data/BaselineED_feat.csv", index=False)

(2, 8)
2  Event-Logs:  ['BPIC15f5' 'BPIC16wm_p']


,log,ratio_variants_per_number_of_traces,ratio_most_common_variant,ratio_top_10_variants,epa_normalized_variant_entropy,epa_normalized_sequence_entropy,epa_normalized_sequence_entropy_linear_forgetting,epa_normalized_sequence_entropy_exponential_forgetting
0,BPIC16wm_p,0.002882,0.295803,0.714106,0.000000,0.00000,0.00000,0.00000
1,BPIC15f5,0.997405,0.001730,0.102076,0.648702,0.60326,0.34241,0.40458


In [8]:
bpic_stats = bpic_features.describe().transpose()
normalized_feature_names = bpic_stats[(bpic_stats['min']>=0)&(bpic_stats['max']<=1)].index.to_list() 
normalized_feature_names = ['ratio_variants_per_number_of_traces', 'ratio_most_common_variant', 
                            'ratio_top_10_variants', 'epa_normalized_variant_entropy', 'epa_normalized_sequence_entropy', 
                            'epa_normalized_sequence_entropy_linear_forgetting', 'epa_normalized_sequence_entropy_exponential_forgetting']
print(normalized_feature_names)
bpic_features[['log']+normalized_feature_names]

['ratio_variants_per_number_of_traces', 'ratio_most_common_variant', 'ratio_top_10_variants', 'epa_normalized_variant_entropy', 'epa_normalized_sequence_entropy', 'epa_normalized_sequence_entropy_linear_forgetting', 'epa_normalized_sequence_entropy_exponential_forgetting']


,log,ratio_variants_per_number_of_traces,ratio_most_common_variant,ratio_top_10_variants,epa_normalized_variant_entropy,epa_normalized_sequence_entropy,epa_normalized_sequence_entropy_linear_forgetting,epa_normalized_sequence_entropy_exponential_forgetting
0,BPIC16wm_p,0.002882,0.295803,0.714106,0.000000,0.00000,0.00000,0.00000
1,BPIC15f5,0.997405,0.001730,0.102076,0.648702,0.60326,0.34241,0.40458


In [9]:
#Features between 0 and 1: 
def write_generator_bpic_experiment(objectives, n_para_obj=2):
    parameters_o = "objectives, "
    experiments = eval(f"[exp for exp in list(itertools.product({(parameters_o*n_para_obj)[:-2]})) if exp[0]!=exp[1]]")
    experiments = list(set([tuple(sorted(exp)) for exp in experiments]))
    for exp in experiments:
        experiment_path = os.path.join('..','data', 'BaselineED_feat')
        os.makedirs(experiment_path, exist_ok=True)
        experiment_path = os.path.join(experiment_path, f"{len(exp)}_{abbrev_obj_keys(exp)}.csv") 


        first_dir = os.path.split(experiment_path[3:])[-1].replace(".csv","")
        second_dir = first_dir.replace("grid_","").replace("objectives","")

        experiment = [
          {
            'pipeline_step': 'event_logs_generation',
            'output_path':'output/generated',
            'generator_params': {
              "experiment": {"input_path": "data/BaselineED_feat.csv",
                "objectives": exp},
              'config_space': {
                'mode': [5, 20],
                'sequence': [0.01, 1],
                'choice': [0.01, 1],
                'parallel': [0.01, 1],
                'loop': [0.01, 1],
                'silent': [0.01, 1],
                'lt_dependency': [0.01, 1],
                'num_traces': [10, 10001],
                'duplicate': [0],
                'or': [0]
              },
              'n_trials': 200
            }
          },
          {
            'pipeline_step': 'feature_extraction',
            'input_path': os.path.join('output', 'features', 'generated', 'BaselineED_feat', first_dir),
            'input_path': os.path.join('output', 'generated', 'BaselineED_feat', first_dir),
            'feature_params': {'feature_set':['simple_stats', 'trace_length', 'trace_variant', 'activities', 'start_activities', 'end_activities', 'eventropies', 'epa_based']},
            'feature_params': {"feature_set":["ratio_variants_per_number_of_traces","ratio_most_common_variant","ratio_top_10_variants","epa_normalized_variant_entropy","epa_normalized_sequence_entropy","epa_normalized_sequence_entropy_linear_forgetting","epa_normalized_sequence_entropy_exponential_forgetting"]},
            'output_path': 'output/plots',
            'real_eventlog_path': 'data/BaselineED_feat.csv',
            'plot_type': 'boxplot'
          },
          {
            "pipeline_step": "benchmark_test",
            "benchmark_test": "discovery",
            "input_path": os.path.join('output', 'generated', 'BaselineED_feat', first_dir),
            "output_path":"output",
            "miners" : ["heu", "imf", "ilp"]
          }
        ]

        output_path = os.path.join('..', 'config_files','algorithm','BaselineED_feat')
        os.makedirs(output_path, exist_ok=True)
        output_path = os.path.join(output_path, f'generator_{os.path.split(experiment_path)[-1].split(".")[0]}.json') 

        with open(output_path, 'w') as f:
            json.dump(experiment, f, ensure_ascii=False)
        print(f"Saved experiment config in {output_path}")
        return experiment


def create_objectives_grid(objectives, n_para_obj=2):
    parameters_o = "objectives, "
    experiments = eval(f"[exp for exp in list(itertools.product({(parameters_o*n_para_obj)[:-2]})) if exp[0]!=exp[1]]")
    experiments = list(set([tuple(sorted(exp)) for exp in experiments]))
    print(len(experiments))
    
    for exp in experiments:
        write_generator_bpic_experiment(objectives=exp)
        
exp_test = create_objectives_grid(normalized_feature_names, n_para_obj=2)        
print(exp_test)

21
Saved experiment config in ../config_files/algorithm/BaselineED_feat/generator_2_enself_rmcv.json
Saved experiment config in ../config_files/algorithm/BaselineED_feat/generator_2_rt10v_rvpnot.json
Saved experiment config in ../config_files/algorithm/BaselineED_feat/generator_2_enself_rvpnot.json
Saved experiment config in ../config_files/algorithm/BaselineED_feat/generator_2_enve_rmcv.json
Saved experiment config in ../config_files/algorithm/BaselineED_feat/generator_2_enve_rvpnot.json
Saved experiment config in ../config_files/algorithm/BaselineED_feat/generator_2_enself_rt10v.json
Saved experiment config in ../config_files/algorithm/BaselineED_feat/generator_2_ense_enve.json
Saved experiment config in ../config_files/algorithm/BaselineED_feat/generator_2_enseef_enve.json
Saved experiment config in ../config_files/algorithm/BaselineED_feat/generator_2_ense_rmcv.json
Saved experiment config in ../config_files/algorithm/BaselineED_feat/generator_2_enve_rt10v.json
Saved experiment con

## Single objective from real logs
(Feature selection)

In [10]:
def write_single_objective_experiment(experiment_path, objectives=["ratio_top_20_variants", "epa_normalized_sequence_entropy_linear_forgetting"]):
    first_dir = os.path.split(experiment_path[3:])[-1].replace(".csv","")
    second_dir = first_dir.replace("grid_","").replace("objectives","")

    experiment = [
      {
        'pipeline_step': 'event_logs_generation',
        'output_path':os.path.join('output','generated', 'grid_1obj'),
        'generator_params': {
          "experiment": {"input_path": experiment_path[3:],
            "objectives": objectives},
          'config_space': {
            'mode': [5, 20],
            'sequence': [0.01, 1],
            'choice': [0.01, 1],
            'parallel': [0.01, 1],
            'loop': [0.01, 1],
            'silent': [0.01, 1],
            'lt_dependency': [0.01, 1],
            'num_traces': [10, 10001],
            'duplicate': [0],
            'or': [0]
          },
          'n_trials': 200
        }
      },
      {
        'pipeline_step': 'feature_extraction',
        'input_path': os.path.join('output','features', 'generated', 'grid_1obj', first_dir, second_dir),
        'feature_params': {'feature_set':['simple_stats', 'trace_length', 'trace_variant', 'activities', 'start_activities', 'end_activities', 'eventropies', 'epa_based']},
        'feature_params': {"feature_set":["ratio_variants_per_number_of_traces","ratio_most_common_variant","ratio_top_10_variants","epa_normalized_variant_entropy","epa_normalized_sequence_entropy","epa_normalized_sequence_entropy_linear_forgetting","epa_normalized_sequence_entropy_exponential_forgetting"]},
        'output_path': 'output/plots',
        'real_eventlog_path': 'data/BaselineED_feat.csv',
        'plot_type': 'boxplot'
      },
      {
        "pipeline_step": "benchmark_test",
        "benchmark_test": "discovery",
        "input_path": os.path.join('output', 'generated', 'grid_1obj', first_dir, second_dir),
        "output_path":"output",
        "miners" : ["heu", "imf", "ilp"]
      }
    ]

    #print("EXPERIMENT:", experiment)
    output_path = os.path.join('..', 'config_files','algorithm','grid_experiments')
    os.makedirs(output_path, exist_ok=True)
    output_path = os.path.join(output_path, f'generator_{os.path.split(experiment_path)[-1].split(".")[0]}.json') 
    with open(output_path, 'w') as f:
        json.dump(experiment, f, ensure_ascii=False)
    print(f"Saved experiment config in {output_path}")
    
    return experiment

def create_objectives_grid(objectives, n_para_obj=2):
    parameters_o = "objectives, "
    if n_para_obj==1:
        experiments = [[exp] for exp in objectives]
    else:
        experiments = eval(f"[exp for exp in list(itertools.product({(parameters_o*n_para_obj)[:-2]})) if exp[0]!=exp[1]]")
    experiments = list(set([tuple(sorted(exp)) for exp in experiments]))
    print(len(experiments), "experiments: ", experiments)
    
    parameters = "np.around(np.arange(0, 1.1,0.1),2), "
    tasks = eval(f"list(itertools.product({(parameters*n_para_obj)[:-2]}))")
    tasks = [(f'task_{i+1}',)+task for i, task in enumerate(tasks)]
    print(len(tasks))
    for exp in experiments:
        df = pd.DataFrame(data=tasks, columns=["task", *exp])
        experiment_path = os.path.join('..','data', 'grid_experiments')
        os.makedirs(experiment_path, exist_ok=True)
        experiment_path = os.path.join(experiment_path, f"grid_{len(df.columns)-1}objectives_{abbrev_obj_keys(exp)}.csv") 
        df.to_csv(experiment_path, index=False)
        print(f"Saved experiment in {experiment_path}")
        write_single_objective_experiment(experiment_path, objectives=exp)
    #df.to_csv(f"../data/grid_{}objectives_{abbrev_obj_keys(objectives.tolist())}.csv" ,index=False)
        
exp_test = create_objectives_grid(normalized_feature_names, n_para_obj=1)        
print(exp_test)

7 experiments:  [('ratio_top_10_variants',), ('ratio_variants_per_number_of_traces',), ('epa_normalized_variant_entropy',), ('epa_normalized_sequence_entropy_linear_forgetting',), ('epa_normalized_sequence_entropy_exponential_forgetting',), ('epa_normalized_sequence_entropy',), ('ratio_most_common_variant',)]
11
Saved experiment in ../data/grid_experiments/grid_1objectives_rt10v.csv
Saved experiment config in ../config_files/algorithm/grid_experiments/generator_grid_1objectives_rt10v.json
Saved experiment in ../data/grid_experiments/grid_1objectives_rvpnot.csv
Saved experiment config in ../config_files/algorithm/grid_experiments/generator_grid_1objectives_rvpnot.json
Saved experiment in ../data/grid_experiments/grid_1objectives_enve.csv
Saved experiment config in ../config_files/algorithm/grid_experiments/generator_grid_1objectives_enve.json
Saved experiment in ../data/grid_experiments/grid_1objectives_enself.csv
Saved experiment config in ../config_files/algorithm/grid_experiments/gen